## Glassnode Endpoint Scrape and API Call
Glassnode contains many useful metrics for bitcoin and ethereum.  They have an API to access these endpoints, but only one request can be made at a time and only so many requests can be made a minute.  Additionaly, a link must be copied and pasted from the API documentation to the code to call the API for each metric.  With nearly 300 metrics, this is a cumbersome process.  

The goal of this task is to scrape all of the endpoints from the glassnode API documentation and identify wether they are for BTC or ETH.  Then, call the API for each of the coin's endpoints at a rate that will not terminate the connection.  

The output of this function is a dataset with daily coin metrics going back to 2010.  This dataset can be used for machine learning algorithms to learn and predict future BTC prices.

In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re
import json

pd.set_option('display.max_colwidth', -1)

<ipython-input-1-d6413d0924b3>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
#define function to convert date to unix
def date_to_unix(
        year=2010,
        month=1,
        day=15,
):
    time = int((dt.datetime(year, month, day, 0, 0, 0).timestamp()))
    return time

In [3]:
#Endpoint Scraping

#Create list of urls to scrape for endpoints
urls = {}
coin = {}
pages = {
         'addresses' : 'https://docs.glassnode.com/api/addresses',
         'blockchain' : 'https://docs.glassnode.com/api/blockchain',
         'distribution' : 'https://docs.glassnode.com/api/distribution',
         'entities' : 'https://docs.glassnode.com/api/entities',
         'eth2' : 'https://docs.glassnode.com/api/eth2',
         'fees' : 'https://docs.glassnode.com/api/fees',
         'indicators' : 'https://docs.glassnode.com/api/indicators',
         'institutions' : 'https://docs.glassnode.com/api/institutions',
         'market' : 'https://docs.glassnode.com/api/market',
         'mining' : 'https://docs.glassnode.com/api/mining',
         'protocols' : 'https://docs.glassnode.com/api/protocols',
         'defi' : 'https://docs.glassnode.com/api/defi',
         'supply' : 'https://docs.glassnode.com/api/supply',
         'transactions' : 'https://docs.glassnode.com/api/transactions'
         }
#access each url and scrape for endpoints 
for page in pages.keys():
    response = requests.get(pages[page])
    content = response.content
    parser = BeautifulSoup(content, 'html.parser')
    text = parser.text
    #extract endpoint
    string = r"(/v1/metrics/[a-z]+/\S+[a-z\d])[A-Z]"
    string2 = r"a=([A-Z]{3})&m=[a-z]+"        
    urls[page] = re.findall(string, text)
    coin[page] = re.findall(string2, text)

#place endpoints in dictionary with appropriate variable name
endpoints = {}
for page in pages.keys():
    for index, url in enumerate(urls[page]):
        var = re.findall(r"/v1/metrics/[a-z]+/(.+)", url)[0]
        endpoints[var] = [url, coin[page][index], page]
urls = [endpoints[x][0] for x in endpoints.keys()]
coins = [endpoints[x][1] for x in endpoints.keys()]
pages = [endpoints[x][-1] for x in endpoints.keys()]
var = endpoints.keys()
endpoints = pd.DataFrame({'feature' : var, 'coin' : coins, 'urls' : urls, 'page' : pages})
#make correction for 'adjusted90' urls (remove 90 from url)
endpoints['urls'] = endpoints['urls'].str.replace('adjusted90', 'adjusted')
endpoints.to_csv('model_output/gn_endpoints.csv')
endpoints

,feature,coin,urls,page
0,sending_to_exchanges_count,BTC,/v1/metrics/addresses/sending_to_exchanges_count,addresses
1,receiving_from_exchanges_count,BTC,/v1/metrics/addresses/receiving_from_exchanges_count,addresses
2,count,BTC,/v1/metrics/transactions/count,transactions
3,sending_count,BTC,/v1/metrics/entities/sending_count,entities
4,receiving_count,BTC,/v1/metrics/entities/receiving_count,entities
...,...,...,...,...
286,transfers_to_exchanges_count,BTC,/v1/metrics/transactions/transfers_to_exchanges_count,transactions
287,transfers_from_exchanges_count,BTC,/v1/metrics/transactions/transfers_from_exchanges_count,transactions
288,transfers_volume_to_exchanges_sum,BTC,/v1/metrics/transactions/transfers_volume_to_exchanges_sum,transactions
289,transfers_volume_from_exchanges_sum,BTC,/v1/metrics/transactions/transfers_volume_from_exchanges_sum,transactions


In [4]:
import requests as req
from datetime import date
today = date.today()

#Define function to access API and request data
def get_metrics(coin, endpoints = endpoints, start = [2010, 9, 1], until = [today.year, today.month, today.day]):
        #filter to coins
        endpoints = endpoints[endpoints['coin'] == coin]
        #Initiate API parameters and dictionary for each coins response
        coins_data = {}
        
        #call api
        api_key='1t0KVebVMNMdGk149JtLxFIfBFk',  # Insert your own API key here, this one is only illustrative.
        asset=coin,
        status=False,
        headers=False,
        resolution='24h',
        wait=10
        
        # convert dates to unix
        s = date_to_unix(year=start[0], month=start[1], day=start[2])
        u = date_to_unix(year=until[0], month=until[1], day=until[2])
        
        #loop through metrics and request data.  Store in dictionary
        for index, row in endpoints.iterrows():
            feature = row[0]
            url = row[2]
            response = req.get(
                f'https://api.glassnode.com{url}',
                {
                    'api_key': api_key,
                    'a': asset,
                    's': s,
                    'u': u,
                    'i': resolution
                },
                timeout=wait)
            print(index, feature)
            try:
                response.json()[0]['o'] 
                print('nested')
                data = response.json()
                result = pd.json_normalize(data)
            
            except KeyError:
                print('not nested')
                result= pd.DataFrame.from_dict(response.json())
            
            col_dict = {x:'{}'.format(feature) for x in result.columns}
            col_dict['t'] = 'date'
            result = pd.DataFrame(result.rename(columns = col_dict))        
            if status:
                print(response)
            time.sleep(2)
            print(result.head())
            if index != 0:
                coin_response = pd.merge(left = coin_response, right = result, on = 'date', how = 'outer')
            if index == 0:
                coin_response = result
        return coin_response

In [5]:
#define coin
coin = 'BTC'
data = get_metrics(coin)


0 sending_to_exchanges_count
not nested
<Response [200]>
         date  sending_to_exchanges_count
0  1283299200  0                         
1  1283385600  0                         
2  1283472000  0                         
3  1283558400  0                         
4  1283644800  0                         
1 receiving_from_exchanges_count
not nested
<Response [200]>
         date  receiving_from_exchanges_count
0  1283299200  0                             
1  1283385600  0                             
2  1283472000  0                             
3  1283558400  0                             
4  1283644800  0                             
2 count
not nested
<Response [200]>
         date  count
0  1283299200  223  
1  1283385600  165  
2  1283472000  155  
3  1283558400  93   
4  1283644800  146  
3 sending_count
not nested
<Response [200]>
         date  sending_count
0  1283299200  95           
1  1283385600  102          
2  1283472000  85           
3  1283558400  64           
4  

29 min_1_usd_count
not nested
<Response [200]>
         date  min_1_usd_count
0  1283299200  103388         
1  1283385600  103416         
2  1283472000  103592         
3  1283558400  103718         
4  1283644800  103812         
30 min_10_usd_count
not nested
<Response [200]>
         date  min_10_usd_count
0  1283299200  1028            
1  1283385600  1044            
2  1283472000  1040            
3  1283558400  1052            
4  1283644800  1046            
31 min_100_usd_count
not nested
<Response [200]>
         date  min_100_usd_count
0  1283299200  290              
1  1283385600  296              
2  1283472000  294              
3  1283558400  300              
4  1283644800  302              
32 min_1k_usd_count
not nested
<Response [200]>
         date  min_1k_usd_count
0  1283299200  36              
1  1283385600  36              
2  1283472000  36              
3  1283558400  36              
4  1283644800  36              
33 min_10k_usd_count
not nested
<Respons

61 balance_miners_all
not nested
<Response [200]>
         date  \
0  1283299200   
1  1283385600   
2  1283472000   
3  1283558400   
4  1283644800   

                                                 balance_miners_all  
0  {'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1387903.0300007}  
1  {'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1387352.8400006}  
2  {'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1391052.8300006}  
3  {'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1394252.7800006}  
4  {'Genesis': 50, 'Patoshi': 1096245.49, 'other': 1396107.3600006}  
62 balance_miners_change
not nested
<Response [200]>
         date  balance_miners_change
0  1283299200  72038.010001         
1  1283385600  71053.040001         
2  1283472000  70853.070001         
3  1283558400  70853.080001         
4  1283644800  71157.700001         
63 balance_otc_desks
not nested
<Response [200]>
         date  balance_otc_desks
0  1534291200  23.041882        
1  1534377600  748.045418       
2  1

95 cdd_supply_adjusted
not nested
<Response [200]>
         date  cdd_supply_adjusted
0  1283299200  0.077338           
1  1283385600  0.035928           
2  1283472000  0.111588           
3  1283558400  0.029303           
4  1283644800  0.570383           
96 cdd_supply_adjusted_binary
not nested
<Response [200]>
         date  cdd_supply_adjusted_binary
0  1283299200  1                         
1  1283385600  0                         
2  1283472000  1                         
3  1283558400  0                         
4  1283644800  1                         
97 average_dormancy_supply_adjusted
not nested
<Response [200]>
         date  average_dormancy_supply_adjusted
0  1283299200  2.689328e-06                    
1  1283385600  1.034287e-06                    
2  1283472000  3.737482e-06                    
3  1283558400  9.758926e-07                    
4  1283644800  4.157050e-06                    
98 spent_output_price_distribution_ath
not nested
<Response [200]>
         d

103 sopr_less_155
not nested
<Response [200]>
         date  sopr_less_155
0  1283299200  0.945752     
1  1283385600  0.973730     
2  1283472000  0.965733     
3  1283558400  0.977781     
4  1283644800  1.017811     
104 sopr_more_155
not nested
<Response [200]>
         date  sopr_more_155
0  1293062400  2.744538     
1  1293148800  2.744538     
2  1293235200  2.744538     
3  1293321600  2.744538     
4  1293408000  2.744538     
105 hodler_net_position_change
not nested
<Response [200]>
         date  hodler_net_position_change
0  1283299200  258809.807559             
1  1283385600  254671.974913             
2  1283472000  253257.261538             
3  1283558400  251179.070878             
4  1283644800  245933.571939             
106 hodled_lost_coins
not nested
<Response [200]>
         date  hodled_lost_coins
0  1283299200  3.535932e+06     
1  1283385600  3.543056e+06     
2  1283472000  3.550393e+06     
3  1283558400  3.559231e+06     
4  1283644800  3.561412e+06     
1

...

182 purpose_etf_holdings_sum
not nested
<Response [200]>
         date  purpose_etf_holdings_sum
0  1613692800  6036.701599             
1  1613779200  6036.701599             
2  1613865600  6036.701599             
3  1613952000  8288.620000             
4  1614038400  9320.010000             
183 purpose_etf_flows_sum
not nested
<Response [200]>
         date  purpose_etf_flows_sum
0  1613779200  0.000000             
1  1613865600  0.000000             
2  1613952000  2251.918401          
3  1614038400  1031.390000          
4  1614124800  327.087000           
184 qbtc_holdings_sum
not nested
<Response [200]>
         date  qbtc_holdings_sum
0  1612310400  23208.872047     
1  1612396800  23320.568444     
2  1612483200  23363.646244     
3  1612569600  23363.646244     
4  1612656000  23363.646245     
185 qbtc_flows_sum
not nested
<Response [200]>
         date  qbtc_flows_sum
0  1612396800  1.116964e+02  
1  1612483200  4.307780e+01  
2  1612569600  0.000000e+00  
3  161265600

218 inflation_rate
not nested
<Response [200]>
         date  inflation_rate
0  1283299200  1.090965      
1  1283385600  1.078119      
2  1283472000  1.153982      
3  1283558400  1.327524      
4  1283644800  1.156708      
219 active_24h
not nested
<Response [200]>
         date    active_24h
0  1283299200  29693.889999
1  1283385600  27696.720000
2  1283472000  25056.810000
3  1283558400  24775.550000
4  1283644800  78154.110000
220 active_1d_1w
not nested
<Response [200]>
         date   active_1d_1w
0  1283299200  110752.550001
1  1283385600  112837.140000
2  1283472000  110228.450000
3  1283558400  112019.410000
4  1283644800  119660.160000
221 active_1w_1m
not nested
<Response [200]>
         date  active_1w_1m
0  1283299200  315621.82   
1  1283385600  314282.73   
2  1283472000  318618.33   
3  1283558400  313745.93   
4  1283644800  308471.41   
222 active_1m_3m
not nested
<Response [200]>
         date  active_1m_3m
0  1283299200  892723.83   
1  1283385600  888854.50   
2

245 lth_profit_sum
not nested
<Response [200]>
         date  lth_profit_sum
0  1283299200  2.158064e+06  
1  1283385600  2.164448e+06  
2  1283472000  2.169989e+06  
3  1283558400  2.176386e+06  
4  1283644800  2.182687e+06  
246 lth_sth_profit_loss_relative
nested
<Response [200]>
         date  lth_sth_profit_loss_relative  lth_sth_profit_loss_relative  \
0  1283299200  2.537029e-07                  0.556130                       
1  1283385600  1.796125e-07                  0.556657                       
2  1283472000  3.236719e-07                  0.556907                       
3  1283558400  2.547235e-07                  0.557255                       
4  1283644800  3.250076e-07                  0.557690                       

   lth_sth_profit_loss_relative  lth_sth_profit_loss_relative  
0  0.102238                      0.269111                      
1  0.073657                      0.338691                      
2  0.107331                      0.259089                    

264 transfers_volume_adjusted_sum
not nested
<Response [200]>
         date  transfers_volume_adjusted_sum
0  1283299200  28757.409997                 
1  1283385600  34736.700000                 
2  1283472000  29856.340000                 
3  1283558400  30026.360000                 
4  1283644800  137208.600000                
265 transfers_volume_adjusted_mean
not nested
<Response [200]>
         date  transfers_volume_adjusted_mean
0  1283299200  128.956996                    
1  1283385600  210.525455                    
2  1283472000  192.621548                    
3  1283558400  322.864086                    
4  1283644800  946.266207                    
266 transfers_volume_adjusted_median
not nested
<Response [200]>
         date  transfers_volume_adjusted_median
0  1283299200  10.99                           
1  1283385600  50.15                           
2  1283472000  101.15                          
3  1283558400  99.75                           
4  1283644800  50.05    

288 transfers_volume_to_exchanges_sum
not nested
<Response [200]>
         date  transfers_volume_to_exchanges_sum
0  1313539200  0.007500                         
1  1313625600  8.538937                         
2  1313712000  39.591418                        
3  1313798400  0.030000                         
4  1313884800  1.819000                         
289 transfers_volume_from_exchanges_sum
not nested
<Response [200]>
         date  transfers_volume_from_exchanges_sum
0  1313539200  0.000000                           
1  1313625600  0.489908                           
2  1313712000  0.000000                           
3  1313798400  8.078029                           
4  1313884800  0.000000                           
290 transfers_volume_within_exchanges_sum
not nested
<Response [200]>
         date  transfers_volume_within_exchanges_sum
0  1313539200  0.000000                             
1  1313625600  7.559029                             
2  1313712000  0.000000              

In [6]:
data['date'] = pd.to_datetime(data['date'], unit = 's')

In [7]:
#save data
data.to_csv('model_output/BTC_{}'.format(today.strftime('%Y%m%d')))
